In [16]:
# Corpus from https://www.kaggle.com/datasets/chazzer/big-language-detection-dataset?resource=download

In [45]:
import pandas as pd
import numpy as np
import regex as re
import warnings
warnings.simplefilter("ignore")

Preprocess corpus data in bulk

In [46]:
topLang = ["eng", "cmn", "hin", "spa", "fra", "ara", "ben", "rus", "por", "ind", "urd", "deu", "jpn", "swh", "pnb", "tam", "kor", "vie", "jav", "ita", "tha", "tgl", "pol", "yor", "ukr", "ibo", "npi", "ron", "nld", "zsm", "afr", "grc", "swe", "heb", "lat", "san", "gle", "mri", "chr", "nav", "haw", "smo"]

data = pd.read_csv("sentences.csv")

data.drop(["id"], axis=1, inplace=True)
data = data[data["lan_code"].isin(topLang)]

data["sentence"] = data.apply(lambda x: re.sub(r"(?=[\p{Common}])[^']|(?<![a-zA-Z])'|'(?![a-zA-Z])", " ", x["sentence"].lower()), axis=1)

""" To enforce all languages have an equal number of samples:"""
# data = filtered_data.groupby("lan_code").sample(28)

""" To just ensure that language sample sizes are not larger than 1000:"""
data = data[data.groupby("lan_code").cumcount() < 1000]

data.to_csv("language_corpus.csv", encoding='utf-8', index=False)

In [8]:
data = pd.read_csv("language_corpus.csv")

# Encode the labels and retrieve the sentence data
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(data["lan_code"])

text_data = data["sentence"]

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

train_text, test_text, train_labels, test_labels = train_test_split(text_data, labels, test_size=0.2)

cv = CountVectorizer(ngram_range=(1,4))
cv.fit(train_text)

train_text = cv.transform(train_text)
test_text = cv.transform(test_text)

In [44]:
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import accuracy_score

model = ComplementNB()
model.fit(train_text, train_labels)

predictions = model.predict(test_text)

ac = accuracy_score(test_labels, predictions)

print("accuracy: " + str(ac))

accuracy: 0.8334636434714621
